# Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain & Vector Search
> *This notebook should work well with the **`conda_python3`** kernel in SageMaker Studio on ml.t3.medium instance*

---

Previously, we used the Anthropic Claude model in Amazon Bedrock to demonstrate a basic Question Answering (QA) system, and learned the value of grounding a model with additional context before generating a response. In the previous notebook, we had to manually provide the model with relevant data and context ourselves. However, this approach is not fit for enterprise-level QA systems where there could be hundreds of thousands of large documents.

## Retrieval Augmented Generation (RAG)

We can improve upon this process by implementing an architecture called retrieval augmented generation (RAG). RAG retrieves data from outside the LLM's training data sources and augments the prompts by adding the relevant retrieved data as context. RAG extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, without needing to retrain the model. It is a cost-effective approach to improving LLM output so it remains relevant, accurate, and useful in various contexts.

## Solution

In this notebook, we augment LLM responses to user queries by implementing RAG using context from external documents. First, we process documents and store these into a vector store. Next, we search the vector store using the user's question, and return relevant data as external context to the LLM. Finally, the LLM generates an answer to the user's question based on the new context provided.

We will walk through implementing the following two patterns: Question Answering (QA) and Conversational AI with conversation memory. 

Let’s break down the solution a little further. 

### Prepare documents for search
![Documents](./images/embeddings_lang.png)

First, the documents must be processed and then indexed in a vector store.
- Load the documents from our directory
- Process the documents by splitting them into smaller chunks
- Create a numerical vector representation of each chunk using an embeddings model
- Create an index using the chunks and the corresponding embeddings

### Respond to the user’s question
![Question](./images/chatbot_lang.png)

Once the vector store is indexed with documents and embeddings, we can search for text relevant to the question being asked. The relevant chunks are sent to the model as additional context, where the model will then generate the answer.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved

Let's get started!

## Setup

In [ ]:
import sys
import os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

In [ ]:
required_models = [
    "amazon.titan-embed-text-v1",
    "amazon.titan-embed-text-v2:0",
    "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    "amazon.rerank-v1:0"
]
validate_model_access(required_models)

In [ ]:
import json
import boto3
import warnings
from pathlib import Path
from rich import print as rprint
from rich.markdown import Markdown
warnings.filterwarnings('ignore')


boto3_bedrock = boto3.client("bedrock-runtime")

## Configure LangChain

LangChain provides convenient integrations with Amazon Bedrock and other services like vector stores and retrievers. We begin with instantiating the large language model (LLM) and the embeddings model. We are using Anthropic Claude models for text generation and Amazon Titan Embeddings G1 - Text for text embedding.

Note: Amazon Bedrock offers a choice of high-performing foundation models (FMs). You can replace the value for `model_id` with one of the available [model IDs](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html) as follows. Some models have different requirements for inputs such as prompt format. As of this writing, all models are supported in the US West (Oregon, us-west-2) Region. If you are using another AWS Region, check the latest [model support by AWS Region](https://docs.aws.amazon.com/bedrock/latest/userguide/models-regions.html).

```python
llm = ChatBedrock(model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0", ...)
```


In [ ]:
from langchain_aws.embeddings import BedrockEmbeddings
from langchain_aws.chat_models import ChatBedrock
from langchain.load.dump import dumps

# Instantiate the LLM

model_id = "us.anthropic.claude-3-5-haiku-20241022-v1:0"

llm = ChatBedrock(
    model_id=model_id,
    model_kwargs={"max_tokens": 500}
)

# Instantiate the Amazon Titan Embeddings G1 - Text embeddings model
bedrock_embeddings = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-text-v2:0" # change this model ID to use another embeddings model
)

## Usecase Introduction - Model Risk and Model Governance Assistant
In this notebook we will learn the application of RAG through a practical example. The use case we will be working on is a Model Risk and Model Governance Assistant. This assistant will help users understand the risks associated with deploying machine learning models in production. The assistant will provide information on the following topics:
- Model Risk Management
- Model Governance
- Regulatory Compliance
- Model Monitoring
- Model Validation
- And more

We will use some publicly available regulatory guideline documents to serve as the source for our RAG solution. You can vew the documents in the `../data/model_risk` directory.

## Data Preparation
We will load the documents with the help of [PyPDF in LangChain](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf).

We will utilize a few different techniques when loading the documents that will help improve the retrieval quality.

#### Outline based splitting
By default LangChain's `PyPDFLoader` will break each document up into pages. We could then potentially use a chunking strategy such as `RecursiveCharacterTextSplitter` to further break down the pages into smaller chunks. 
However, this could lead to suboptimal results if the most relevant information we are looking for is split across multiple pages. Instead, we will split the documents into sections based on the documents own table of contents. The implementation for this approach is provided in the rag_utils.outline_parser module [(source)](./rag_utils/outline_parser.py).
Note that this approach only works on PDFs that contain a table of contents.


In [ ]:
from langchain.document_loaders import PyPDFLoader
from rag_utils.outline_parser import PyPDFOutlineParser

docs_path = Path("../data/model_risk")
doc_files = list(docs_path.glob("*.pdf"))

section_chunks = []

for doc_path in doc_files:
    loader = PyPDFLoader(file_path=doc_path.as_posix())
    loader.parser = PyPDFOutlineParser()
    sections = loader.load()
    for sec in sections:
        sec.metadata.update({"file": doc_path.name})
    
    section_chunks += sections
    

Each section chunk now contains a the contents and metadata associated with that section

Now let's test out our embedding model on a single section to see what an embedding looks like below. These embeddings could be generated for the entire corpus of documents and stored in a vector store for easy retrieval.

In [ ]:
sample_embedding = bedrock_embeddings.embed_query(section_chunks[0].page_content)
modelId = bedrock_embeddings.model_id
rprint("Embedding model Id :", modelId)
rprint("Sample embedding of a document chunk: ", sample_embedding[:10])
rprint("Size of the embedding: ", len(sample_embedding))


## Create the vector store
In this workshop we will use the ***vector engine for Amazon OpenSearch Serverless.***

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application — without impacting data ingestion.

The code block below will provision an OpenSearch Serverless collection and create the vector store that we will use to store the embeddings.

In [ ]:
from langchain.vectorstores import OpenSearchVectorSearch
from opensearchpy import RequestsHttpConnection
from rag_utils.oss_utils import get_aws_auth, get_host

auth = get_aws_auth()
host = get_host("bedrock-workshop-rag")
host = f"{host}:443"

index_name = "model-risk-index"

vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    embedding_function=bedrock_embeddings,
    opensearch_url=host,
    http_auth=auth,
    timeout = 7200,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    engine="faiss",
)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time

child_splitter = RecursiveCharacterTextSplitter(
    separators=["\n", "\n\n"], chunk_size=2000, chunk_overlap=250
)

split_docs = child_splitter.split_documents(section_chunks)

if vector_db.client.indices.exists(index=index_name):
    rprint("Index already exists. Checking if documents are already added.")
    doc_count = vector_db.client.count(index=index_name)["count"]
    if doc_count != len(split_docs):
        rprint(f"Index contains less than {len(split_docs)} documents. Adding documents.")
        vector_db.delete_index()
        vector_db.add_documents(split_docs)
    else:
        rprint(f"Index contains {doc_count} documents. No need to add documents.")

else:
    rprint("Creating index and adding documents.")
    vector_db.add_documents(split_docs)
    rprint("Documents added to the index. Waiting for verification")
    doc_count = vector_db.client.count(index=index_name)["count"]
    max_wait = 120
    while doc_count != len(split_docs):
        rprint("Documents not yet reflected in index. Waiting for 20 seconds.")
        time.sleep(20)
        doc_count = vector_db.client.count(index=index_name)["count"]
        max_wait -= 20
        if max_wait <= 0:
            rprint("Documents not reflected in index in 120 seconds. Exiting.")
            break
    rprint(f"Documents added to the index. Index contains {doc_count} documents.")

## Searching the vector store
Let's explore the various ways that we can query the vector store.

### Semantic search methods
[Semantic search](https://opensearch.org/docs/latest/search-plugins/semantic-search/) considers the context and intent of a query. In OpenSearch, semantic search is facilitated by neural search with text embedding models. Semantic search creates a dense vector (a list of floats) and ingests data into a k-NN index.

Short for k-nearest neighbors, the k-NN plugin enables users to search for the k-nearest neighbors to a query point across an index of vectors.  To determine the neighbors, you can specify the space (the distance function) you want to use to measure the distance between points. We will explore some of the distance functions later in this lab.

The k-NN plugin supports three different methods for obtaining the k-nearest neighbors from an index of vectors:
- Approximate k-NN
- Script Score k-NN
- Painless extensions

#### Approximate k-NN search
Standard k-NN search methods compute similarity using a brute-force approach that measures the nearest distance between a query and a number of points, which produces exact results. This works well in many applications. However, in the case of extremely large datasets with high dimensionality, this creates a scaling problem that reduces the efficiency of the search. [Approximate k-NN search](https://opensearch.org/docs/latest/search-plugins/knn/approximate-knn/) methods can overcome this by employing tools that restructure indexes more efficiently and reduce the dimensionality of searchable vectors. Using this approach requires a sacrifice in accuracy but increases search processing speeds appreciably.

Of the three search methods the k-NN plugin provides, this method offers the best search scalability for large datasets. This approach is the preferred method when a dataset reaches hundreds of thousands of vectors.

In [ ]:
# Search query
query = "What can be considered a model?"

# Search for the 3 most relevant documents
results = vector_db.similarity_search(query, k=3)

rprint(dumps(results, pretty=True))

#### Exact k-NN with scoring script
The k-NN plugin implements the OpenSearch [score script](https://opensearch.org/docs/latest/search-plugins/knn/knn-score-script/) plugin that you can use to find the exact k-nearest neighbors to a given query point.

Because the score script approach executes a brute force search, it doesn’t scale as well as the approximate approach. This approach is preferred for searches over smaller bodies of documents or when a pre-filter is needed. Using this approach on large indexes may lead to high latencies.

In [ ]:
results = vector_db.similarity_search(
    query, 
    k=2,
    is_appx_search=False,
    search_type="script_scoring"
)

rprint(dumps(results, pretty=True))

#### k-NN Painless Scripting extensions (exact)
Currently, the vector engine for Amazon OpenSearch serverless supports the approximate k-NN search and scoring script search methods. Below is an example of the [painless scripting](https://opensearch.org/docs/latest/search-plugins/knn/painless-functions/) search method on an Amazon OpenSearch service for your reference.

Similar to the k-NN Script Score, you can use this method to perform a brute force, exact k-NN search across an index. This approach has slightly slower query performance compared to the k-NN scoring script. If your use case requires more customization over the final score, you should use this approach over k-NN scoring script.

```python
results = docsearch.similarity_search(
    query, 
    is_appx_search=False,
    search_type="painless_scripting"
)

print(dumps(results, pretty=True))
```

#### Exact k-NN search with filters
You can apply [k-NN search with filters](https://opensearch.org/docs/latest/search-plugins/knn/filter-search-knn/) with either the  scoring script or painless extension search methods. Filters can greatly reduce the number of vectors to be searched. Using the k-NN score script, you can apply a filter on an index before executing the nearest neighbor search (sometimes referred to as a pre-filter search). This is useful for dynamic search cases where the index body may vary based on other conditions.

In [ ]:
query = "What are the acceptable model evaluation techniques?"

# filter on a specific document
pre_filter = {"bool": {"filter": {"match": {"metadata.file": "sr1107a1.pdf"}}}}

# Pre-filter results
results = vector_db.similarity_search(
    query, 
    k=2,
    is_appx_search=False,
    search_type="script_scoring",
    pre_filter=pre_filter   
)

rprint(dumps(results, pretty=True))

#### Spaces - similarity or distance measures

Approximate Search through OpenSearch supports the following similarity or distance measures:

**Cosine similarity** – The cosine of the angle between two vectors in a vector space.

**Euclidean distance** – The straight-line distance between points.

**L1 (Manhattan) distance** – The sum of the differences of all of the vector components. L1 distance measures how many orthogonal city blocks you need to traverse from point A to point B.

**L-infinity (chessboard) distance** – The number of moves a King would make on an n-dimensional chessboard. It’s different than Euclidean distance on the diagonals—a diagonal step on a 2-dimensional chessboard is 1.41 Euclidean units away, but 2 L-infinity units away.

**Inner product** – The product of the magnitudes of two vectors and the cosine of the angle between them. Usually used for natural language processing (NLP) vector similarity.

We can specify the distance measure in the `space_type` parameter when we load our documents as seen below.

```python
docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    bedrock_embeddings,
    opensearch_url=host,
    http_auth=auth,
    timeout = 100,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=index_name,
    engine="faiss",
    space_type="l2", # Options are: “l2”, “l1”, “cosinesimil”, “linf”, “innerproduct”; default: “l2”
)

#### [Optional] Engines and algorithms
The Approximate k-NN search methods leveraged by OpenSearch use approximate nearest neighbor (ANN) algorithms from the [NMSLIB](https://github.com/nmslib/nmslib), [FAISS](https://github.com/facebookresearch/faiss), and [Lucene](https://lucene.apache.org/) libraries to power k-NN search.

The engine details are as follows:

- Non-Metric Space Library (NMSLIB) – NMSLIB implements the HNSW ANN algorithm
- Facebook AI Similarity Search (FAISS) – FAISS implements both HNSW and IVF ANN algorithms
- Lucene – Lucene implements the HNSW algorithm

Each of the three engines used for approximate k-NN search has its own attributes that make one more sensible to use than the others in a given situation. In general, NMSLIB and FAISS should be selected for large-scale use cases. Lucene is a good option for smaller deployments, but offers benefits like smart filtering where the optimal filtering strategy—pre-filtering, post-filtering, or exact k-NN—is automatically applied depending on the situation.

We can specify the engine as shown below.

Note: As of this writing, Amazon OpenSearch Serverless vector search collections don't support the Apache Lucene ANN engine. Vector search collections only support the HNSW algorithm with FAISS and do not support IVF and IVFQ. Please check the updated [limitations](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/serverless-vector-search.html#serverless-vector-limitations).

```python
docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    bedrock_embeddings,
    opensearch_url=host,
    http_auth=auth,
    timeout = 100,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=index_name,
    engine="faiss", # Options are: “nmslib”, “faiss”, “lucene”; default: “nmslib”
)
```

**For HNSW, we can tune the m, ef_construction, and ef_search parameters to achieve our desired trade-off:**

**m** – Controls the maximum number of edges a node in a graph can have. Because each node has to store all of its edges, increasing this value will increase the memory footprint, but also increase the connectivity of the graph, which will improve recall.

**ef_construction** – Controls the size of the candidate queue for edges when adding a node to the graph. Increasing this value will increase the number of candidates to consider, which will increase the index latency. However, because more candidates will be considered, the quality of the graph will be better, leading to better recall during search.

**ef_search** – Similar to ef_construction, it controls the size of the candidate queue for graph traversal during search. Increasing this value will increase the search latency, but will also improve the recall.

In general, we chose configurations that gradually increase the parameters, as detailed in the following table.

![](./images/hnsw_parameters.png)

Below is an example using the **FAISS** engine and providing a parameter configuration that provides a balance between latency, memory, and recall (see table above).

```python
docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    bedrock_embeddings,
    opensearch_url=host,
    http_auth=auth,
    timeout = 100,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=index_name,
    engine="faiss",
    space_type="l2",
    m=16, # maximum number of edges
    ef_construction=128, # size of the candidate queue for edges
    ef_search=128 # size of the candidate queue for graph traversal
)
```

### Maximum marginal relevance search (MMR)
If you’d like to look up for some similar documents, but you’d also like to receive diverse results, MMR is a method you should consider. Maximal marginal relevance optimizes for similarity to query AND diversity among selected documents. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples.

In [ ]:
# we fetch 10 results but then return the top 3 most diverse
results = vector_db.max_marginal_relevance_search(query, k=3, fetch_k=10)

rprint(dumps(results, pretty=True))

## Orchestrating RAG using LangChain
Now that we can query our vector database for documents, we can retrieve data from outside of a large language model's training data sources and augment our prompts by adding the relevant retrieved data in context.

We can use LangChain to build applications that read data from stored internal documents and summarize them into conversational responses. We can create a Retrieval Augmented Generation (RAG) workflow that introduces new information to the language model during prompting. Implementing context-aware workflows like RAG reduces model hallucination and improves response accuracy.

### Single turn generative question answering

Let's start with a simple example where given a user query we retrieve relevant documents from the vector store and use the retrieved documents as context to generate a response.

We'll construct a prompt template that will take the user's question and the retrieved documents as context and generate a response.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

template = """Answer the question based only on the following context. 
If the context does not provide sufficient information to answer the question, politely indicate that you are unable to assist. 
Only answer questions related to model risk and model governance.

<context>
{context}
</context>

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

retriever = vector_db.as_retriever(k=5)

# in the first step we retrieve the context and pass through the input question
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
    
)

# In the subsequent steps pass the context and question to the prompt, send the prompt to the llm and parse the output as a string
chain = setup_and_retrieval | prompt | llm | output_parser

Let’s try this with our earlier queries:

In [ ]:
query = "What can be considered a model?"

response = chain.invoke(query)
rprint(response)

In [ ]:
query = "What are some acceptable model evaluation techniques?"

response = chain.invoke(query)
rprint(response)

## Improving the RAG workflow
The example provides a naive implementation of RAG. While it works for demonstration purposes, it has a number of limitations:
- The are no guardrails in place to ensure that the model does not engage in any topics other than model risk and governance
- The user's question may not be the most optimal search query for the vector store.
- The workflow is stateless and not conversational. User is not able to ask follow up questions
- Retrieval is based purely on semantic similarity. There is no filtering or ranking of the retrieved documents.

We'll address these limitations by introducing a few more components to our workflow.
- [**LangGraph**](https://www.langchain.com/langgraph): A python library for building custom LLM and Agent based workflows. LangGraph allows us to express a workflow as a finite state machine (FSM) making it possible to orchestrate LLM driven steps, hardcoded logic, and human input in the loop workflows.
- [**BM25 Retrieval**](https://en.wikipedia.org/wiki/Okapi_BM25): A retrieval method that uses a probabilistic model to rank documents based on their relevance to a query. BM25 is a popular choice for information retrieval tasks and can be used as a complement to semantic search methods.
- [**Hybrid Retrieval**](https://python.langchain.com/docs/how_to/ensemble_retriever/): Will combine the BM25 and FAISS retrievers to provide a more robust retrieval method.
- [**Amazon Reran Model**](https://docs.aws.amazon.com/bedrock/latest/userguide/rerank.html): A model that can be used to re-rank the retrieved documents based on their relevance to the query. This will help us filter out any irrelevant documents and improve the quality of the context provided to the LLM.
- [**Bedrock Session API**](https://docs.aws.amazon.com/bedrock/latest/userguide/sessions.html): A new API that allows us to maintain a session with the LLM. This will help us maintain conversation state and allow for follow up questions.

In [ ]:
from pydantic import BaseModel, Field
from typing import Annotated, Literal

from typing_extensions import TypedDict

from langchain_core.messages import HumanMessage, AIMessage

from langchain_aws.document_compressors.rerank import BedrockRerank
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever

from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from rag_utils.oss_utils import OpenSearchBM25Retriever

Let's start with the improvements to the retrieval workflow. We'll start by creating a new retriever that combines the BM25 and FAISS retrievers. We'll then use this retriever to retrieve documents based on the user's query. Finally, we'll use the Rerank model to re-rank the retrieved documents based on their relevance to the query.

In [ ]:
# used for lexical search
bm_25_retriever = OpenSearchBM25Retriever(client=vector_db.client, index_name=index_name, k=10)

# used for semantic search
semantic_search_retriever = vector_db.as_retriever(search_kwargs={"k": 10})

# combine results from both retrievers
# since k was set to 10 for both retrievers, the fusion retriever will return 20 results
fusion_retriever = EnsembleRetriever(
    retrievers=[semantic_search_retriever, bm_25_retriever], weights=[0.5, 0.5]
)

bedrock_agent_runtime = boto3.client("bedrock-agent-runtime")

# reranker model
reranker = BedrockRerank(
    model_arn="arn:aws:bedrock:us-west-2::foundation-model/amazon.rerank-v1:0",
    client=bedrock_agent_runtime,
    region_name="us-west-2",
    top_n=5, # the top 5 ranked results from the fusion retriever will be returned
)

# rerank the combined results from the fusion retriever
rerank_retriever = ContextualCompressionRetriever(
    base_compressor=reranker, base_retriever=fusion_retriever
)

In [ ]:
# test the improved retriever
query = "What are some acceptable model evaluation techniques?"
results = rerank_retriever.invoke(query)
rprint(results)

Next we can start defining our workflow using LangGraph. We'll start by defining the states and transitions for our workflow. We'll then define the steps that will be executed

In [ ]:
# State is often defined a TypedDict
class RAGState(TypedDict):
    messages: Annotated[list, add_messages] # contains the conversation history, new messages are appended to the list
    search_query: str                       # the search query generated by the model
    context_summary: str                    # summary of the conversation   
    search_results: list                    # the search results from the retriever
    input_validated: bool                   # whether the input has been validated or not


# these data models are used to define the outputs of the different steps in the pipeline

# output of the input validation step
class InputValidator(BaseModel):
    input_validated: bool = Field(description="Whether the input is validated or not.")

# output of the search query generation step
class SearchQuery(BaseModel):
    query: str = Field(description="The search query to be used for retrieval.")
    context_summary: str = Field(description="The summary of the context of what is being asked.")

Nodes in the workflow can be defined as functions. Each function takes in the current state, performs some operations, and returns the updated state. Think of the state as a car being assembled in a factory. Each function is a step in the assembly line that takes the car from one state to another. The final state is the fully assembled car, ready to be driven off the lot.

In [ ]:
# a simple chain to validate if the users input is on topic
# this can be extended to include more complex validation logic
def validate_user_input(state: RAGState) -> Literal["formulate_search_query", "generate_answer"]:
    """
    Validate the user input to ensure it is a valid search query.
    """


    system_prompt = "You are the first line of defense for a chatbot that answers questions about model risk and model governance regulations. " \
                    "Your job is to validate the user's input and ensure it is relevant to the topic." \
                    "Use the available tool to validate the input and return the result." 
    
    validation_chain = llm.bind_tools(tools=[InputValidator]) | PydanticToolsParser(tools=[InputValidator])

    result = validation_chain.invoke([("system", system_prompt)] + state["messages"])[0]

    # choose the next step based on the validation result
    if result.input_validated:
        # if valid input, we can proceed to the next step
        return "formulate_search_query"
    else:
        # if invalid input, we can jump to the last step where a canned response is provided
        return "generate_answer"

In [ ]:
# given the conversation context, this step generates to help get better search results
def formulate_search_query(state: RAGState) -> RAGState:
    """Formulate the search query based on the user question."""
    
    query = state["messages"][-1].content
    
    prompt = "You are assisting with matters related to model risk and model governance. " \
            "You have access to a comprehensive database of regulations and best practices" \
            "as they relate to model risk and model governance. " \
            "Based on your expertise, help refine the user's question into a search query that can be used to retrieve relevant information." \
            "Given the prior interaction, please provide a search query that is relevant to the topic." \
            "Additionally, if the user's question is a follow-up question based on information from previous interactions, please take that into account." \
            "Please provide a brief summary of the context of what is being asked." \
            "Use the available tool to record the search query." \

    query_generation_chain  = llm.bind_tools([SearchQuery]) | PydanticToolsParser(tools=[SearchQuery])
    response = query_generation_chain.invoke([("system", prompt)] + state["messages"])
    query = response[0].query
    context_summary = response[0].context_summary

    # the updated state is returned as a dictionary 
    return {"search_query": query, "input_validated": True, "context_summary": context_summary}


In [ ]:
# perform the search using the search query generated in the previous step
def search(state: RAGState) -> RAGState:
    """Perform a search using the formulated search query."""
    
    search_query = state["search_query"]
    print("Search query: ", search_query)
    results = rerank_retriever.invoke(search_query)
    return {"search_query": search_query, "search_results": results}

In [ ]:
def generate_answer(state: RAGState) -> RAGState:
    """Generate an answer based on the search results."""

    # return the canned response if the input is not valid
    if state["input_validated"] == False:
        return {
            "messages": [AIMessage(content="I'm sorry but I cannot assist with that.")],
        }

    search_results = state["search_results"]
    if not search_results:
        return {
            "messages": [AIMessage(content="I'm sorry I could not find any relevant information.")],
        }

    search_results = "\n".join([doc.page_content for doc in search_results])
    question = state["messages"][-1].content
    context_summary = state["context_summary"]

    prompt = f"""Answer the question based only on the following context. 
    If the context does not provide sufficient information to answer the question, politely indicate that you are unable to assist. 
    Only answer questions related to model risk and model governance.

    <search_results>
    {search_results}
    </search_results>

    The context in which the question is being asked is:
    <context_summary>
    {context_summary}
    </context_summary>


    <question>
    {question}
    </question>
    """

    response_message = llm.invoke(prompt)
    return {
        "messages": [response_message],
        "search_query": state["search_query"],
        "search_results": state["search_results"],
        "input_validated": False,
        "context_summary": context_summary
    }

Now that all of the components are in place, we can start defining our workflow. We'll start by defining the nodes in the workflow, followed by the transitions between the nodes.

In [ ]:
graph_builder = StateGraph(RAGState)

graph_builder.add_node("formulate_search_query", formulate_search_query)
graph_builder.add_node("search", search)
graph_builder.add_node("generate_answer", generate_answer)

graph_builder.add_conditional_edges(START, validate_user_input)
graph_builder.add_edge("formulate_search_query", "search")
graph_builder.add_edge("search", "generate_answer")
graph_builder.add_edge("generate_answer", END)

To save the session state, we will use the [Bedrock Session API](https://docs.aws.amazon.com/bedrock/latest/userguide/sessions-opensource-library.html). The session API allows us to maintain a session with the LLM and keep track of the conversation state. This will help us maintain context across multiple turns of conversation.

The open source [langgraph-checkpoint-aws](https://github.com/langchain-ai/langchain-aws/tree/main/libs/langgraph-checkpoint-aws) library provides a convenient abstraction for the Bedrock Session API that works directly with LangGraph.

In [ ]:
from langgraph_checkpoint_aws.saver import BedrockSessionSaver

memory = BedrockSessionSaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
# display the graph
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))

except Exception as e:
    rprint("Issue with mermaid rendering API")
    mermaid_code = graph.get_graph().draw_mermaid()
    rprint(
        Markdown(
            f"To view the graph, copy the code below and paste it into a [mermaid live editor](https://mermaid.live/edit)"
        )
    )
    rprint(mermaid_code)

In [ ]:
# create a new session id to store the state of the graph
session_id = bedrock_agent_runtime.create_session()["sessionId"]

In [ ]:
config = {"configurable": {"thread_id": session_id},}
user_input = "What are the best practices for model governance?"
res = graph.invoke({"messages": [{"role": "user", "content": user_input}], "input_validated": False}, config)
rprint(res["messages"][-1].content)

In [ ]:
# ask a question that is not related to model risk
user_input = "How fast is a cheetah?"
res = graph.invoke({"messages": [{"role": "user", "content": user_input}], "input_valid": False}, config)
rprint(res["messages"][-1].content)

In [ ]:
user_input = "What are some example risk that I should document for a credit risk model?"
res = graph.invoke({"messages": [{"role": "user", "content": user_input}], "input_valid": False}, config)
rprint(res["messages"][-1].content)

In [ ]:
# follow up on the previous question
user_input = "How about for a fraud detection model?"
res = graph.invoke({"messages": [{"role": "user", "content": user_input}], "input_valid": False}, config)
rprint(res["messages"][-1].content)

### Clean up
You have reached the end of this workshop. Following cell will delete all created resources.


In [ ]:
vector_db.delete_index()
rprint("Index deleted.")

## Conclusion
In the above implementation of RAG based Question Answering and Conversational AI, we have explored the following concepts and how to implement them using the LangChain integrations for Amazon Bedrock and a local vector store:

- Loading documents and processing them into smaller chunks
- Creating a vector store using FAISS
- Generating embeddings with an embeddings model
- Searching the vector store to retrieve context relevant to the question
- Performing Generative Question Answering using foundation models
- Improving trust in our system by providing citations with every answer
- Preparing prompt templates to use as input to the LLM
- Storing conversation memory and providing the history as context to the LLM

### Next steps
- Experiment with different vector stores
- Leverage various text and embedding models available through Amazon Bedrock to see alternate outputs
- Explore options such as persistent storage of embeddings and document chunks
- Use Amazon Bedrock Knowledge Bases, a fully managed RAG capability with built-in session context management

# Thank You